# **2.11 Generating Molecules with MolGAN**

MolGAN : GAN(Generative Adversarial Network)를 이용한 모델

Generator : 무작위 노이즈를 입력받아 한번에 분자 그래프를 그려냄

Discriminator : Generator가 만든 분자와 실제 데이터셋을 비교하며 진짜인지 가짜인지 판별

Reward Network : Reinforcement Learining을 사용해 생성된 분자가 우리가 원하는 특정 화학적 성질을 갖도록 유도

R-GCN(Relational-GCN) : 입력받은 그래프를 해석하기 위해 R-GCN사용, 일반적인 GCN과 다르게 결합의 종류를 구분해 학습 가능



In [ ]:
# Download from MolNet
# Try tox21 or LIPO dataset
tasks, datasets, transformers = dc.molnet.load_tox21()
df = pd.DataFrame(data={'smiles': datasets[0].ids})

num_atoms = 12

feat = dc.feat.MolGanFeaturizer(max_atom_count=num_atoms, atom_labels=[0, 5, 6, 7, 8, 9, 11, 12, 13, 14])

smiles = data['smiles'].values
Filter out the molecules with too many atoms to reduce the number of unnecessary error messages in later steps.

filtered_smiles = [x for x in smiles if Chem.MolFromSmiles(x).GetNumAtoms() < num_atoms]

# featurize molecules
features = feat.featurize(filtered_smiles)

데이터셋 선택: Tox 21

고정된 크기의 행렬을 생성하기 때문에 최대 원자수 정하기
->  허용원자 종류 feat

smiles를 Node Mtrix와 Adjacency Tensor로 나눔

In [ ]:
# create model
gan = MolGAN(learning_rate=ExponentialDecay(0.001, 0.9, 5000), vertices=num_atoms)
dataset = dc.data.NumpyDataset([x.adjacency_matrix for x in features],[x.node_features for x in features])

gan.fit_gan(iterbatches(25), generator_steps=0.2, checkpoint_interval=5000)
generated_data = gan.predict_gan_generator(1000)

피처라이제이션 실패한 데이터들 indices로 제거

학습률 감쇠 및 데이터셋 구축

In [ ]:
nmols = feat.defeaturize(generated_data)
print("{} molecules generated".format(len(nmols)))

nmols = list(filter(lambda x: x is not None, nmols))

# currently training is unstable so 0 is a common outcome
print ("{} valid molecules".format(len(nmols)))

De-featurization

Defeaturize : 행렬데이터를 RDkit이 이해가능하게 분자객체(nmol)로 바꿈

Validity Check: 말이 안되는 분자들 필터링

중복제거

# **2.12 Introduction to GROVER**

ChemBERTa : 텍스트 기반 transformer

GROVER(Graph Representation frOm-superVised mEssage passing tRansformer) : GNN & Transformer 결합



GNN과 Transformer 하이브리드 구조

Self-Supervised Learnig : 레이블 없는 대규모 분자 데이터에서 규칙 학습

Node/Edge level : 원자와 결합 특징

Graph level : 분자 전체의 통계적 특징(예: 작용기 유무)




표현력 극대화/ 풍부한 문맥 이해

Semantic Motif 추출 : 화학적으로 의미있는 Functional Groups단위로 인식
-> NLP처럼 Vocabulary 구축 -> 데이터 분할 및 병렬화


Pre training -> Fine tuning -> Feature Extraction -> Prediction

💡 **2.11 MolGAN & 2.12 GROVER**

Graph to Graph: Smiles를 거치지 않고 행렬로 찍어냄

Reward Network : 특정목표 달성 가능

/

Pretraining & fine-tuning : 이미 학습된 모델을 이용해 맞춤 파인튜닝 가능

Gnn & transformer : 미시적 거시적 맥락 모두 파악 가능